In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import random
import io
import json

In [2]:
from datetime import datetime
log_path ="D:/P2023/LOG/resnet18_50t_test2_0915.txt"
# print and fprint at the same time
def pprint(output = '\n' , filename = log_path, show_time = False):
    print(output)
    with open(filename, 'a') as f:
        if show_time:
            f.write(datetime.now().strftime("[%Y-%m-%d %H:%M:%S] "))

        f.write(str(output))
        f.write('\n')
pprint("test", show_time=True)

test


In [3]:
mean = np.array([0.5, 0.5, 0.5])
std = np.array([0.25, 0.25, 0.25])

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

In [4]:
import torch
import random
from torchvision import datasets, transforms

# Set your data directory
data_dir = "D:/P2023/DATA/tvgh"

# Create the merged dataset
merged_dataset = datasets.ImageFolder(data_dir, data_transforms['train'])

# Count the number of samples in each class
class_counts = {}
for _, label in merged_dataset.samples:
    class_name = merged_dataset.classes[label]
    class_counts[class_name] = class_counts.get(class_name, 0) + 1
print(class_counts)
# Determine the minimum number of samples among "MGN" and "IGA" classes
min_class_count = min(class_counts["MGN"], class_counts["IGA"])
print(min_class_count)
# Create lists to hold the indices of samples to keep and to drop
indices_to_keep = [min_class_count - ii -1 for ii in range(min_class_count*2)]

merged_dataset.samples = [merged_dataset.samples[i] for i in indices_to_keep]
merged_dataset.targets = [merged_dataset.targets[i] for i in indices_to_keep]

{'IGA': 7311, 'MGN': 993}
993


In [5]:
from collections import Counter
print(Counter(merged_dataset.targets))

Counter({0: 993, 1: 993})


In [6]:
test_loader = torch.utils.data.DataLoader(merged_dataset, batch_size=4, shuffle=True, num_workers=0)
print(test_loader.dataset)

Dataset ImageFolder
    Number of datapoints: 1986
    Root location: D:/P2023/DATA/tvgh
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=warn)
               CenterCrop(size=(224, 224))
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.5 0.5 0.5], std=[0.25 0.25 0.25])
           )


In [7]:
dataset_sizes = len(merged_dataset)
class_names = merged_dataset.classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(class_names)
print(device)

['IGA', 'MGN']
cuda:0


In [8]:
def test_mod(model):
    since = time.time()
    model.eval()   # Set model to evaluate mode

    running_loss = 0.0
    running_corrects = 0
    
    tp_positive = 0
    fp_positive = 0
    tn_negative = 0
    fn_negative = 0
    positive_other = 0
    negative_other = 0
    
    # Iterate over data.
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        
        # MN -> positive
        # iga -> negative
        tp_positive += torch.sum((preds == 1) & (labels.data == 1))
        fp_positive += torch.sum((preds == 1) & (labels.data == 0))
        tn_negative += torch.sum((preds == 0) & (labels.data == 0))
        fn_negative += torch.sum((preds == 0) & (labels.data == 1))
        positive_other += torch.sum((preds == 2) & (labels.data == 1))
        negative_other += torch.sum((preds == 2) & (labels.data == 0))

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]
        
        epoch_recallmn = tp_positive.double() / (tp_positive + fn_negative) # ***
        epoch_recallig = tn_negative.double() / (tn_negative + fp_positive) # ***
                
        epoch_precmn = tp_positive.double() / (tp_positive + fp_positive) # ***
        epoch_precig = tn_negative.double() / (tn_negative + fn_negative) # ***
        
        epoch_f1mn = (2 * epoch_recallmn * epoch_precmn) / (epoch_recallmn + epoch_precmn) # ***
        epoch_f1ig = (2 * epoch_recallig * epoch_precig) / (epoch_recallig + epoch_precig) # ***
        
        # print('{} Loss: {:.4f} Acc: {:.4f} Recall_MGN: {:.4f} Recall_IGAN: {:.4f} Precision_MGN: {:.4f} Precision_IGAN: {:.4f} F1_MGN: {:.4f} F1_IGAN: {:.4f}'.format(
        #     phase, epoch_loss, epoch_acc, epoch_recallmn, epoch_recallig, epoch_precmn, epoch_precig, epoch_f1mn, epoch_f1ig)) # ***
        pprint('{} Loss: {:.4f} Accuracy: {:.4f} \n     Recall Precision F1_score OTHER\n MGN: {:.4f} {:.4f} {:.4f} {}\n IGAN:{:.4f} {:.4f} {:.4f} {}\n'.format(
                phase, epoch_loss, epoch_acc, epoch_recallmn, epoch_precmn, epoch_f1mn, positive_other, epoch_recallig, epoch_precig, epoch_f1ig, negative_other)) # ***
            
    time_elapsed = time.time() - since
    pprint('Testing complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

In [ ]:
def test_mod(PATH):
    model = models.resnet18(weights=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 2)
    model = model.to(device)
    
    # # Step 1: Load the model and state dictionary from the .pt file
    # checkpoint = torch.load(PATH)  # Load the state dictionary from the file
    
    # # Assuming your model is named 'model' and the state dictionary key is 'state_dict'
    # model.load_state_dict(checkpoint)
    
    # # Step 2: Serialize the state dictionary into bytes
    # buffer = io.BytesIO()
    # torch.save(checkpoint, buffer)

    # buffer.seek(0)  # Make sure to reset the buffer position to the beginning
    # checkpoint = torch.load(buffer)
    # model.load_state_dict(checkpoint)
    # # print(checkpoint)
    # # Step 3: The state dictionary is now stored in 'buffer' as bytes and can be used as needed
    model.eval()
    test_mod(model)


path = "D:/P2023/WEIGHT/raw_data_training/res18_50t.pt"
test_mod(path)

C:\Users\User\anaconda3\envs\GPU\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
